# VacationPy 

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import pprint

# Import API key
from api_keys import g_key

#configure gmaps
gmaps.configure(api_key = g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
dfVacation = pd.read_csv("output_data/cities.csv")
dfVacation.dropna(inplace = True)
dfVacation.head()
# sort out the headers for this dataframe (remove Max Temp)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,CityDate,Temp (C)
0,0,Mataura,-46.19,168.86,281.96,76,3,1.99,NZ,1594773906,8.81
1,1,Kapaa,22.08,-159.32,301.15,74,40,5.70,US,1594773689,28.00
2,2,Morgan City,29.70,-91.21,304.26,74,1,3.60,US,1594773906,31.11
3,3,Betioky,-23.72,44.38,286.90,90,0,0.44,MG,1594773907,13.75
4,4,Hermanus,-34.42,19.23,275.93,95,0,3.13,ZA,1594773701,2.78


In [3]:
# set the latitude and longitude
locations = dfVacation[["Lat", "Lng"]]
humidity = dfVacation["Humidity"]
locations.head()

,Lat,Lng
0,-46.19,168.86
1,22.08,-159.32
2,29.70,-91.21
3,-23.72,44.38
4,-34.42,19.23


In [4]:
# ### Humidity Heatmap
# * Configure gmaps.
# * Use the Lat and Lng as locations and Humidity as the weight.
# * Add Heatmap layer to map. 

In [5]:
figure_layout = {
    'width' : '400px',
    'height' : '300px',
    'border' : '1px solid black',
    'padding' : '1px',
    'margin' : '0 auto 0 auto'
}
fig = gmaps.figure(map_type = "HYBRID")
heatmap_layer = gmaps.heatmap_layer(locations,
                                    weights = humidity,
                                    dissipating = True,
                                    max_intensity = 200,
                                    point_radius = 10
                                   )
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria

Narrow down the cities to fit weather conditions.
Drop any rows will null values.

In [27]:
IdealWeather = dfVacation[(dfVacation["Temp (C)"] > 30) & (dfVacation["Temp (C)"] < 40) 
                           & (dfVacation["Humidity"] > 50)
                           & (dfVacation["Cloudiness"] < 40)
                           & (dfVacation["Wind Speed"] < 10)
                         ]
IdealWeather
### Need to update IdealWeather4 -> to IdealWeather

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,CityDate,Temp (C)
2,2,Morgan City,29.70,-91.21,304.26,74,1,3.60,US,1594773906,31.11
59,59,Cabo San Lucas,22.89,-109.91,307.15,66,5,7.70,MX,1594773911,34.00
66,66,Hong Kong,22.29,114.16,305.37,70,27,5.81,HK,1594773829,32.22
76,76,Abu Dhabi,24.47,54.37,305.15,79,0,1.00,AE,1594773905,32.00
110,110,Harlingen,26.19,-97.70,306.15,55,1,9.80,US,1594773816,33.00
132,132,Avera,33.19,-82.53,304.15,52,1,2.60,US,1594773738,31.00
139,139,Chalmette,29.94,-89.96,305.15,70,1,1.50,US,1594773917,32.00
221,221,Kwang Binh,17.48,106.60,304.72,57,0,0.97,VN,1594773925,31.57


### Hotel Map

Store into variable named hotel_df.

Add a "Hotel Name" column to the DataFrame.

Set parameters to search for hotels with 5000 meters.

Hit the Google Places API for each city's coordinates.

Store the first Hotel result into the DataFrame.

Plot markers on top of the heatmap.

In [7]:
hotelDF = IdealWeather4
IdealWeather4["Hotel Name"] = ""
IdealWeather4.head()
IdealWeather4.drop(["Max Temp"], axis = 1)

,City_ID,City,Lat,Lng,Humidity,Cloudiness,Wind Speed,Country,CityDate,Temp (C),Hotel Name
2,2,Morgan City,29.70,-91.21,74,1,3.60,US,1594773906,31.11,
59,59,Cabo San Lucas,22.89,-109.91,66,5,7.70,MX,1594773911,34.00,
66,66,Hong Kong,22.29,114.16,70,27,5.81,HK,1594773829,32.22,
76,76,Abu Dhabi,24.47,54.37,79,0,1.00,AE,1594773905,32.00,
110,110,Harlingen,26.19,-97.70,55,1,9.80,US,1594773816,33.00,
132,132,Avera,33.19,-82.53,52,1,2.60,US,1594773738,31.00,
139,139,Chalmette,29.94,-89.96,70,1,1.50,US,1594773917,32.00,
221,221,Kwang Binh,17.48,106.60,57,0,0.97,VN,1594773925,31.57,


In [8]:
# https://maps.googleapis.com/maps/api/place/nearbysearch/json
# ?location=-33.8670522,151.1957362
# &radius=1500
# &type=restaurant
# &keyword=cruise
# &key=YOUR_API_KEY

In [15]:
# emptyHotelList = []
for index, row in IdealWeather4.iterrows(): 
    vacLat = row["Lat"]
    vacLng = row["Lng"]
    vacCity = row["City"]
    nearbyUrl = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
            "location" : f'{vacLat}, {vacLng}',
            "key" : g_key,
            "radius" : 5000,
            "type" : "lodging",
            "rankBy" : "prominence"
            }
    response = requests.get(nearbyUrl, params)   
    pp = pprint.PrettyPrinter(indent=4)
    myHotel = response.json()
#     pp.pprint(myHotel)
    try:
        closeHotel = myHotel['results'][0]['name']
    except IndexError:
        pass
#     print(close_hotel)
    IdealWeather4.loc[index,"Hotel Name"] = closeHotel
#         emptyHotelList.append(close_hotel)
# print(emptyHotelList)
    
IdealWeather4

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,CityDate,Temp (C),Hotel Name
2,2,Morgan City,29.70,-91.21,304.26,74,1,3.60,US,1594773906,31.11,Hampton Inn & Suites Morgan City
59,59,Cabo San Lucas,22.89,-109.91,307.15,66,5,7.70,MX,1594773911,34.00,Hotel Tesoro Los Cabos
66,66,Hong Kong,22.29,114.16,305.37,70,27,5.81,HK,1594773829,32.22,Mini Hotel Central
76,76,Abu Dhabi,24.47,54.37,305.15,79,0,1.00,AE,1594773905,32.00,Beach Rotana
110,110,Harlingen,26.19,-97.70,306.15,55,1,9.80,US,1594773816,33.00,Courtyard by Marriott Harlingen
132,132,Avera,33.19,-82.53,304.15,52,1,2.60,US,1594773738,31.00,Courtyard by Marriott Harlingen
139,139,Chalmette,29.94,-89.96,305.15,70,1,1.50,US,1594773917,32.00,Marina Motel Chalmette
221,221,Kwang Binh,17.48,106.60,304.72,57,0,0.97,VN,1594773925,31.57,Nam Long Hotel


In [ ]:
gmaps.marker_layer

In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in IdealWeather4.iterrows()]
locations = IdealWeather4[["Lat", "Lng"]]
locations

,Lat,Lng
2,29.70,-91.21
59,22.89,-109.91
66,22.29,114.16
76,24.47,54.37
110,26.19,-97.70
132,33.19,-82.53
139,29.94,-89.96
221,17.48,106.60


In [22]:
points_of_interest = gmaps.marker_layer(locations, info_box_content=hotel_info) 
fig.add_layer(points_of_interest)
fig
# fig.savefig("PointsOfInterest.png", bbox_inches='tight', dpi=450)

Figure(layout=FigureLayout(height='420px'))

In [ ]:
#  # Add marker layer ontop of heat map
# From Assistant Instructor to Everyone:  06:03 PM
# import gmaps

# gmaps.configure(api_key="AI...")

# fig = gmaps.figure()
# embed_minimal_html('export.html', views=[fig])

    
# # Display figure